In [4]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""


"""
Links to be used -->
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#remote-data-yahoo
https://python.hotexamples.com/fr/examples/finsymbols/-/get_nasdaq_symbols/python-get_nasdaq_symbols-function-examples.html
https://cppsecrets.com/users/61421071171099711410397117114971189910411710711764103109971051084699111109/Using-Python-And-Pandas-Datareader-to-Analyze-Financial-Data.php
https://books.google.fr/books?id=4f30DwAAQBAJ&pg=PA640&lpg=PA640&dq=get_nasdaq_symbols&source=bl&ots=qAuJg8ol64&sig=ACfU3U274lgObBYlkO_aCkFYRcrNBYBtZg&hl=fr&sa=X&ved=2ahUKEwir097T18P0AhUJxBQKHSINAN0Q6AF6BAgoEAM
https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html
P152 du cours
"""

#import the library to be used 

import pandas as pd
import pandas_datareader.data as reader
#from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
import datetime as dt
import statsmodels.api as sm
#import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
#import seaborn as sns
#from random import randint
import numpy as np
import xlwings as xw

book = xw.Book("python excel.xlsx")
book.sheets
output_sheet = book.sheets("output_sheet")

#Define fonctions for the caculation of prices, returns and merge 
# a dataframe with the factors and the stock returns 

def prices(ticker,t=5):
    end = dt.datetime.now()
    start = dt.date(end.year - t,end.month,end.day)
    stock_prices = reader.get_data_yahoo(ticker, start, end)['Adj Close']
    stock_prices = stock_prices.resample('1M').last()
    return stock_prices

def returns(ostock_prices):
    stock_returns = ostock_prices.pct_change()
    stock_returns = stock_returns.dropna()
    return stock_returns

def merger_stock(df,ticker,i=5):
    stock_return = returns(prices(ticker,i))
    df_end = df.merge(stock_return, on = 'Date')
    df_end.rename(columns={'Adj Close': ticker}, inplace=True)
    return df_end

In [5]:

#Define fonctions for the Fama French 3 factors

def get_factors_ff3():
    ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
    ff3_monthly.rename(columns={'date_ff_factors':'Date'}, inplace=True)
    ff3_monthly.set_index('Date', inplace = True)
    return ff3_monthly

def get_ff3_premia(factors):
    market_premium = factors['Mkt-RF'].mean()
    size_premium = factors['SMB'].mean()
    value_premium = factors['HML'].mean()
    premia = (market_premium, size_premium, value_premium)
    return premia

def get_dataf_ff3(list_tickers,k=5):
    stock_return = returns(prices(list_tickers[0],k))
    data = get_factors_ff3().merge(stock_return, on = 'Date')
    data.rename(columns={'Adj Close': list_tickers[0]}, inplace=True) 
    for i in range(1, len(list_tickers)):
        data = merger_stock(data,list_tickers[i],k)
    return data

def get_ff3_reg_df(list_tickers,l=5):
    data_df = get_dataf_ff3(list_tickers,l)
    n = len(list_tickers)
    for i in range(0,n):
        excess_ret = data_df[list_tickers[i]] - data_df['RF']
        excess_ret_name = list_tickers[i] + '-RF'
        data_df[excess_ret_name] = excess_ret
    return data_df

def get_ff3_reg_stat(list_tickers,l=5):
    n = len(list_tickers)
    reg_df = get_ff3_reg_df(list_tickers,l)
    X = reg_df[['Mkt-RF', 'SMB', 'HML']]
    reg_stat = pd.DataFrame()
    for i in range(0,n):
        excess_ret_name = list_tickers[i] + '-RF'
        Y = reg_df[excess_ret_name]
        X1 = sm.add_constant(X)
        model = sm.OLS(Y,X1)
        results = model.fit()
        intercept, beta_m, beta_s, beta_v = results.params
        reg_stat[excess_ret_name] = [intercept, beta_m, beta_s, beta_v]
    reg_stat.index = ['const','Mkt-RF', 'SMB', 'HML']
    return reg_stat

def get_ff3_exp_ret_df(list_tickers,l=5):
    RF = get_ff3_reg_df(list_tickers,l)['RF'].mean()
    n = len(list_tickers)
    reg_stat_df = get_ff3_reg_stat(list_tickers,l=5)
    premia_list = get_ff3_premia(get_factors_ff3())
    exp_ret_df = pd.DataFrame()
    exp_ret_df.index = ['Expected return']
    for i in range(0,n):
        excess_ret_name = list_tickers[i] + '-RF'
        exp_ret_monthly = RF + reg_stat_df.loc['Mkt-RF',excess_ret_name] * premia_list[0] + reg_stat_df.loc['SMB',excess_ret_name] * premia_list[1] + reg_stat_df.loc['HML',excess_ret_name] * premia_list[2] 
        exp_ret_annually = exp_ret_monthly * 12 
        exp_ret_df[list_tickers[i]] = exp_ret_annually 
    exp_ret_df.columns = [list_tickers]
    return exp_ret_df


def get_returns_df(list_tickers,k=5):
    stock_return = returns(prices(list_tickers[0],k))
    data = pd.DataFrame()
    data[list_tickers[0]] = stock_return 
    for i in range(1, len(list_tickers)):
        data = merger_stock(data,list_tickers[i],k)
    data = data.iloc[:-2]
    return data


def get_market_exposure_df(list_tickers,l=5):
    n = len(list_tickers)
    data = pd.DataFrame()
    reg_stat_df = get_ff3_reg_stat(list_tickers,l)
    factor_df = get_dataf_ff3(list_tickers,l)
    for i in range(0,n):
        excess_ret_name = list_tickers[i] + '-RF'
        expo_name = list_tickers[i] + '_m_expo'
        exp_ret_monthly = reg_stat_df.loc['Mkt-RF',excess_ret_name] * factor_df['Mkt-RF']
        data[expo_name] = exp_ret_monthly 
    return data

def get_specific_exposure_df(list_tickers,l=5):
    n = len(list_tickers)
    data = pd.DataFrame()
    factor_expo_df = get_factor_exposure_df(list_tickers,l)
    market_expo_df = get_market_exposure_df(list_tickers,l)
    total_expo_df = get_returns_df(list_tickers,l)
    for i in range(0,n):
        expo_name = list_tickers[i] + '_s_expo'
        factor_expo_name = list_tickers[i] + '_f_expo'
        market_expo_name = list_tickers[i] + '_m_expo'
        total_expo_name = list_tickers[i]
        data[expo_name] = total_expo_df[total_expo_name] - market_expo_df[market_expo_name] - factor_expo_df[factor_expo_name]
    return data

def get_factor_exposure_df(list_tickers,l=5):
    n = len(list_tickers)
    data = pd.DataFrame()
    reg_stat_df = get_ff3_reg_stat(list_tickers,l)
    factor_df = get_dataf_ff3(list_tickers,l)
    for i in range(0,n):
        excess_ret_name = list_tickers[i] + '-RF'
        expo_name = list_tickers[i] + '_f_expo'
        exp_ret_monthly = reg_stat_df.loc['SMB',excess_ret_name] * factor_df['SMB'] + reg_stat_df.loc['HML',excess_ret_name] * factor_df['HML']
        data[expo_name] = exp_ret_monthly 
    return data

def get_ranked_df(list_tickers,l=5):
    n = len(list_tickers)
    number_of_date = l*12 - 2
    factor_exposure_df = get_factor_exposure_df(list_tickers,l)
    factor_exposure_matrix = factor_exposure_df.to_numpy()
    ranking_matrix = np.zeros(shape=(number_of_date,n))
    data = factor_exposure_df
    data_end = pd.DataFrame()
    column_name_to_delete = []
    for i in range(0,number_of_date):
        array = factor_exposure_matrix[i] 
        order = array.argsort()[::-1][:n]
        ranks = order.argsort()[::-1][:n]
        ranking_matrix[i] = ranks
    for k in range(0,n):
        column_name = list_tickers[k] + '_rank'
        data[column_name] = ranking_matrix[:,k]
        column_name_to_delete.append(list_tickers[k] + '_f_expo')
    for m in range(0,n):
        column_rank_name = list_tickers[m] + '_rank'
        data_end[column_rank_name] = data[column_rank_name]
    return data_end

def get_tertile_pf_df(list_tickers,l=5):
    n = len(list_tickers)
    number_of_date = l*12 - 2
    factor_exposure_df = get_factor_exposure_df(list_tickers,l)
    factor_exposure_matrix = factor_exposure_df.to_numpy()
    ranking_matrix = np.zeros(shape=(number_of_date,n))
    data = factor_exposure_df
    data_end = pd.DataFrame()
    column_name_to_delete = []
    for i in range(0,number_of_date):
        array = factor_exposure_matrix[i] 
        order = array.argsort()
        ranks = order.argsort()
        lenght = len(ranks)
        for p in range(0,lenght):
            if 10 <= ranks[p] <= 14:
                ranks[p] = 1
            elif 5 <= ranks[p] <= 9:
                ranks[p] = 2
            else:
                ranks[p] = 3
        ranking_matrix[i] = ranks
    for k in range(0,n):
        column_name = list_tickers[k] + '_tertile'
        data[column_name] = ranking_matrix[:,k]
        column_name_to_delete.append(list_tickers[k] + '_f_expo')
    for m in range(0,n):
        column_rank_name = list_tickers[m] + '_tertile'
        data_end[column_rank_name] = data[column_rank_name]
    return data_end



In [6]:

# play mdr 

#List_tickers_4 = ['MSFT','TSLA','AAPL','GOOG','BABA','FB','AMZN','GE','AMD','PFE','JPM','BAC']
#List_tickers_5 = ['MSFT','TSLA','AAPL']
List_tickers = ['MSFT','TSLA','AAPL','ATVI','AMZN','DPZ','F','CCL','VZ','KO','XOM','BAC','AMG','MMM','ADSK']
#List_tickers_2 = generate_ticker_list()

#Generated lists that works 
#List_tickers_3 = ['CFMS','RVACU','CELC','GS','DXR','FMBI','EAOK','BKEM','NMIH','GTHX']


O_reg_df = get_ff3_reg_df(List_tickers,6)
O_reg_tab = get_ff3_reg_stat(List_tickers,6)
O_exp_ret = get_ff3_exp_ret_df(List_tickers,6)
# print(O_reg_df.head())
# print(O_reg_tab.head())
# print(O_exp_ret.head())


O_total_expo = get_returns_df(List_tickers,6)
O_market_expo = get_market_exposure_df(List_tickers,6)
O_factor_expo = get_factor_exposure_df(List_tickers,6)
O_specific_expo = get_specific_exposure_df(List_tickers,6)


# print(O_total_expo.head())
# print(O_market_expo.head())
# print(O_factor_expo.head())
# print(O_specific_expo.head())

O_rank = get_ranked_df(List_tickers,6).head()
# print(O_rank.head)

O_tertile = get_tertile_pf_df(List_tickers,6)
# print(O_tertile.head())



ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [ ]:
output_sheet.range('A1').options(pd.DataFrame, index = True).value = O_tertile 